In [ ]:
import requests # Importa a biblioteca requests para fazer requisições HTTP
from bs4 import BeautifulSoup # Importa a classe BeautifulSoup da biblioteca bs4 para manipular HTML
import csv # Importa a biblioteca csv para ler e escrever arquivos CSV

def pesquisar_produtos_mercado_livre(produto, num_paginas, preco_min, preco_max): # Define a função pesquisar_produtos_mercado_livre com os parâmetros produto, num_paginas, preco_min e preco_max
    url_base = "https://lista.mercadolivre.com.br/{}" # Define a URL base para pesquisar produtos no Mercado Livre
    produtos = [] # Cria uma lista vazia para armazenar os produtos encontrados
    
    for i in range(num_paginas):  # Loop que percorre o número de páginas informado
        url = url_base.format(produto) + f"_Desde_{i*50+1}"  # Formata a URL incluindo o nome do produto e a posição inicial da página
        resposta = requests.get(url)  # Faz uma requisição GET para a URL e armazena a resposta
        soup = BeautifulSoup(resposta.text, "html.parser")  # Cria um objeto BeautifulSoup a partir do texto da resposta HTML
        itens = soup.find_all("li", class_="ui-search-layout__item")  # Encontra todos os elementos li com a classe "ui-search-layout__item" e armazena na variável itens

        for item in itens:  # Loop que percorre cada item da lista de itens encontrados
            titulo = item.find("h2", class_="ui-search-item__title").text.strip()  # Extrai o título do produto e remove espaços em branco
            link = item.find("a", class_="ui-search-link")["href"]  # Extrai o link do produto
            preco_tags = item.find_all("span", class_="price-tag-fraction")  # Encontra todas as tags span com a classe "price-tag-fraction"
            preco_original = preco_tags[0].text.strip()  # Extrai o preço original do produto
            preco_original_float = float(preco_original.replace(".", "").replace(",", "."))  # Converte o preço original para um número float

            if len(preco_tags) > 1:  # Verifica se há mais de uma tag de preço (preço com desconto)
                preco_desconto = preco_tags[1].text.strip()  # Extrai o preço com desconto
            else:
                preco_desconto = preco_original  # Caso não haja desconto, atribui o preço original

            preco_desconto_float = float(preco_desconto.replace(".", "").replace(",", "."))  # Converte o preço com desconto para um número float

            if preco_min <= preco_desconto_float <= preco_max:  # Verifica se o preço com desconto está dentro do intervalo especificado
                produtos.append({"titulo": titulo, "preco_original": preco_original, "preco_desconto": preco_desconto, "link": link})  # Adiciona o produto à lista de produtos

    return produtos  # Retorna a lista de produtos encontrados
def exportar_para_csv(produtos, nome_arquivo): # Define a função exportar_para_csv com os parâmetros produtos e nome_arquivo
    with open(nome_arquivo, mode='w', newline='', encoding='utf-8') as arquivo_csv: # Abre (ou cria) o arquivo CSV no modo de escrita, com codificação UTF-8
        campos = ["titulo", "preco_original", "preco_desconto", "link"] # Define os campos que serão incluídos no arquivo CSV
        escritor = csv.DictWriter(arquivo_csv, fieldnames=campos) # Cria um objeto DictWriter para escrever dicionários no arquivo CSV
        
        escritor.writeheader()  # Escreve o cabeçalho no arquivo CSV com os nomes dos campos
        for produto in produtos:  # Loop que percorre cada produto na lista de produtos
            escritor.writerow(produto)  # Escreve uma linha no arquivo CSV com os dados do produto

produto = input("Digite o produto que você deseja pesquisar: ") # Solicita ao usuário o nome do produto que deseja pesquisar
num_paginas = int(input("Digite o número de páginas que deseja percorrer: ")) # Solicita ao usuário o número de páginas que deseja percorrer
preco_min = float(input("Digite o valor mínimo do produto: ").replace(",", ".")) # Solicita ao usuário o valor mínimo do produto e converte para float
preco_max = float(input("Digite o valor máximo do produto: ").replace(",", ".")) # Solicita ao usuário o valor máximo do produto e converte para float

resultados = pesquisar_produtos_mercado_livre(produto, num_paginas, preco_min, preco_max) # Chama a função pesquisar_produtos_mercado_livre e armazena os resultados na variável resultados
nome_arquivo = "resultados.csv" # Define o nome do arquivo CSV que será criado
exportar_para_csv(resultados, nome_arquivo) # Chama a função exportar_para_csv para escrever os resultados no arquivo CSV

print(f"Os resultados foram exportados para o arquivo '{nome_arquivo}'.") # Exibe uma mensagem informando que os resultados foram exportados para o arquivo CSV